In [ ]:
# BFS Template
from collections import deque
from typing import Optional
class TreeNode:
    def __init__(self, val = 0):
        self.val = val
        self.left = None
        self.right = None

class Solution:
    '''
    @param root: A Tree
    @return: Level order a list of lists of integer
    '''
    def levelOrder(self, root: Optional[TreeNode]) -> list[list[int]]:
        if root is None:
            return []

        queue = deque([root])
        result = []
        
        while queue:
            level = []
            for _ in range(len(queue)):
                node = queue.popleft()
                level.append(node.val)
                if node.left:
                    queue.append(node.left) 
                if node.right:
                    queue.append(node.right)
            
            result.append(level)
        
        return result



In [ ]:
# Serialize and Deserialize Template
class Solution:
    def serialize(self, root):
        if root is None:
            return "{}"

        queue = [root]
        index = 0
        while index < len(queue):
            if queue[index] is not None:
                queue.append(queue[index].left)
                queue.append(queue[index].right)
            index += 1
        
        # 不刪也不會錯
        while queue[-1] is None:
            queue.pop()
        
        return '{%s}' % ','.join([str(node.val) if node is not None else '#'
                                 for node in queue])
    
    def deserialize(self, data):
        data = data.strip('\n')
        
        if data == '{}':
            return None
        
        # Start value
        vals = data[1:-1].split(',')
        
        root = TreeNode(int(vals[0]))
        queue = [root]
        isLeftChild = True
        index = 0
        
        # Since the first val has been assigned as root, val start from vals[1]
        for val in vals[1:]:
            if val is not '#':
                node = TreeNode(int(val))
                if isLeftChild:
                    # 這邊仍然在處理 queue[index]: 根節點root，還需要處理right，所以index還不能增加
                    queue[index].left = node
                else:
                    queue[index].right = node
                queue.append(node)
            
            if not isLeftChild:
                index += 1
            isLeftChild = not isLeftChild
        
        return root
            

In [ ]:
# Clone Graph
# Definition for a Node.
class Node:
    def __init__(self, val = 0, neighbors = None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []

from typing import Optional
from collections import deque

class Solution:
    """
    @param: node: A undirected graph node
    @return: A undirected graph path
    """
    def cloneGraph(self, node: Optional['Node']) -> Optional['Node']:
        # root = node
        
        if node is None:
            return node
        
        # Use BFS algorithm to traverse the graph and get all nodes
        nodes = self.getNodes(node)
        
        # Copy node, store the old -> new mapping information in a has map
        # 建立映射（只建節點，不連邊）
        mapping = {}
        for cur in nodes:
            # 原始值和clone之間的連結，先做每一個的val，還沒做鄰居
            mapping[cur] = Node(cur.val)
        
        # Copy neighbors(edges)
        for cur in nodes:
            new_node = mapping[cur]
            for neighbor in cur.neighbors:
                # 使用 原始值和clone之間的連結 把值抓來
                new_neighbor = mapping[neighbor]
                new_node.neighbors.append(new_neighbor)
        
        return mapping[node] 
        
    
    def getNodes(self, node: Optional[Node]):
        # 把所有起點放到queue裡
        # 並標記為訪問過了
                      # 只有起點
        queue = deque([node])
        visited = set([node])
        
        # 2. while queue 不空
        while queue:
            # pop出queue的第一個點
            # 把所有的鄰居都扔到隊列裡標記為訪問過
            head = queue.popleft()
            for neighbor in head.neighbors:
                # A 比 B 的寫法好，因為如果B還有多行code，會有縮進的問題，主體部分的縮進太長
                if neighbor in visited:
                    continue
                # 這個visited一定要在同層裡面就操作，不然會重複，造成超時
                visited.add(neighbor)
                queue.append(neighbor)
                # B
                # if neighbor not in visited:
                #     visited.add(neighbor)
                #     ...
                #     ...
                #     ...
                #     queue.append(neighbor)
                    
        return visited
        
# if __name__ == '__main__':
#     Node[1].neighbors = [Node[2], Node[4]]
#     Node[2].neighbors = [Node[1], Node[3]]
#     Node[3].neighbors = [Node[2], Node[4]]
#     Node[4].neighbors = [Node[1], Node[3]]
#     sol = Solution()
#     result = sol.cloneGraph(Node[1])
#     print(result)

TypeError: type 'Node' is not subscriptable